# BLU02 - Exercises Notebook

In [1]:
import hashlib # for grading

import os
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

## 1 Read the Oscars data (graded)

In this first exercise, we aim to create a single dataframe, combining all Oscar nominees from all ceremonies.

With a caveat though: **we want to include seasons from the year 1960 onwards**.

In [2]:
def read_year(folder_path, file_name):
    path = os.path.join(folder_path, file_name)
    return pd.read_csv(path, index_col = 0)

def read_nominees(folder_path):
    files = os.listdir(folder_path)
    # Create a list with the name of all files containing annual nominees from
    # 1960 inclusive and onwards (just the filename, no complete path.)
    # files_from_1960: List[str] = ...
    # YOUR CODE HERE
    files_from_1960: List[str] =[file for file in os.listdir(folder_path) if int(file[0:4])>=1960]

    # Create a list with the dataframes
    # nominees_year: List[pd.DataFrame] = ...
    # YOUR CODE HERE
    nominees_year: List[pd.DataFrame] =  [read_year(folder_path, file) for file in files_from_1960]
        
    # Use pd.concat to create a single dataframe.
    # nominees: pd.DataFrame = ...
    # YOUR CODE HERE

    nominees: pd.DataFrame =  pd.concat(nominees_year, ignore_index=True)
    # Drop the column 'ceremony'.
    # nominees = ...
    # YOUR CODE HERE
    nominees = nominees.drop(columns=['ceremony'])
    
    ## Remove missing values.
    # nominees = ...
    # YOUR CODE HERE
    nominees = nominees.dropna()
    return nominees


nominees = read_nominees(os.path.join('data', 'oscars'))

In [3]:
assert nominees['year_ceremony'].min() == 1960
assert nominees['year_ceremony'].max() == 2023
assert nominees.isna().sum().sum() == 0
assert nominees.shape == (7116, 6)

## 2 Read the IMDB Ratings data (graded)

In [4]:
def read_ratings(file_path): 
    # Read the ratings data and drop the 'director', 'star1', 'star2', 'star3', and 'star4' columns.
    # top_rated: pd.DataFrame = ...
    # YOUR CODE HERE
    top_rated: pd.DataFrame = pd.read_csv(file_path)
        
    columns_to_drop = ['director', 'star1', 'star2', 'star3', 'star4']
    top_rated = top_rated.drop(columns=columns_to_drop)
        
    # Please make the necessary changes and convert the 'runtime' column to int
    # YOUR CODE HERE

    top_rated['runtime'] = top_rated['runtime'].str.replace(' min', '').astype(int)    
    ## Remove the lines with no metascore info.
    # YOUR CODE HERE
    top_rated = top_rated.dropna(subset=['metascore'])
    
    return top_rated

top_rated = read_ratings(os.path.join('data','imdb_top_1000.csv'))

In [5]:
assert top_rated.shape == (843, 5)
assert top_rated.runtime.min() == 64
assert top_rated.runtime.max() == 321
assert top_rated.metascore.isna().sum() == 0
assert set(top_rated.columns) == set([
    'film', 'metascore', 'no_votes', 'rating', 'runtime'
])

## 3 Combine Oscars and Ratings data (graded)

Let's combine both dataframes into a single dataset, using an inner join.

In [6]:
nominees.columns, top_rated.columns

(Index(['year_film', 'year_ceremony', 'category', 'name', 'film', 'winner'], dtype='object'),
 Index(['film', 'runtime', 'rating', 'metascore', 'no_votes'], dtype='object'))

In [7]:
# Remember that you want use a column of both dataframes to combine them.
# Join only the nominees of films present on the ratings list
# best_rated_nominees = ...
# YOUR CODE HERE
best_rated_nominees: pd.DataFrame = pd.merge(nominees, top_rated, how='inner', on='film')

In [8]:
assert best_rated_nominees.shape == (1919, 10)
assert set(best_rated_nominees.columns) == set(['year_film', 'year_ceremony', 'category', 'name', 'film', 'winner',
       'runtime', 'rating', 'metascore', 'no_votes'])

## 4 Read top grossing and budget films data (graded)

We will read the two remaining pieces of data. 

Again, albeit the step-by-step description, we encourage you to use method chaining.

In [9]:
def read_gross(file_path):
    # Read the works data.
    # top_grossing: pd.DataFrame = ...
        # Remove the year column.
    # gross: pd.DataFrame = ...

    ## Ensure that the gross data is read as int.
     # top_grossing: pd.DataFrame = ...
      # YOUR CODE HERE
    top_grossing = (pd.read_csv(file_path)
                    # Ensure that the 'gross' data is read as int
                    .assign(gross=lambda df: df['gross'].str.replace(',', '').astype(int))
                    # Remove the 'year' column
                    .drop(columns=['year'])
                    )
    return top_grossing


def read_budget(file_path):
    # Read the top budget data and drop the 'runtime', 'theaters', and 'year' Columns

    ## Please make the necessary changes and convert the 'budget_rank' column to int
    # YOUR CODE HERE
    top_budget = (pd.read_csv(file_path)
                  .drop(columns=['runtime', 'theaters', 'year'])
                  # Remove non-numeric characters and convert 'budget_rank' to int
                  .assign(budget_rank=lambda df: df['budget_rank'].str.replace('#', '').astype(int))
                  )
    return top_budget


top_grossing = read_gross('data/gross_top_200.csv')
top_budget = read_budget('data/budget_top_500.csv')

In [10]:
assert top_grossing.shape == (200, 3)
assert set(top_grossing.columns) == set([
    'gross_rank', 'film', 'gross'
])

assert top_budget.shape == (500, 3)
assert set(top_budget.columns) == set([
   'budget_rank', 'film', 'production_cost'
])

## 5 Combine the top grossing and budget films

Like we did for Oscar nominees and top rated films, now we combine the top budget and grossing films.

In [11]:
# Combine both dataframes, again using an inner type of join
# top_grossing_budget : pd.DataFrame = ....
# YOUR CODE HERE
def combine_grossing_and_budget(grossing_df, budget_df):
    # Combine dataframes using an inner join
    top_grossing_budget = pd.merge(grossing_df, budget_df, how='inner', on='film')
    
    return top_grossing_budget


top_grossing_budget = combine_grossing_and_budget(top_grossing, top_budget)


In [12]:
assert top_grossing_budget.shape == (129, 5)
assert set(top_grossing_budget.columns) == set(
    [
        'budget_rank', 'film', 'production_cost', 'gross_rank', 'gross'
    ]
)

## 6 Combine everything (graded)

The final goal here is to create a single dataframe.

In [13]:
best_rated_nominees.columns, top_grossing_budget.columns

(Index(['year_film', 'year_ceremony', 'category', 'name', 'film', 'winner',
        'runtime', 'rating', 'metascore', 'no_votes'],
       dtype='object'),
 Index(['gross_rank', 'film', 'gross', 'budget_rank', 'production_cost'], dtype='object'))

In [14]:
def get_common_columns(df1, df2):
    """
    Identify common columns in two dataframes.

    Parameters:
    df1 (pd.DataFrame): The first dataframe.
    df2 (pd.DataFrame): The second dataframe.

    Returns:
    List[str]: A list of common column names.
    """
    # Get the set of columns for each dataframe
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    # Find the intersection (common columns) between the two sets
    common_columns = list(columns_df1.intersection(columns_df2))

    return common_columns

# Usage example:
# common_cols = get_common_columns(dataframe1, dataframe2)
# print(common_cols)


In [15]:
# Combine best_rated_nominees and nyp into a single dataframe.
# You need to figure out the common column shared between the two dataframes
# top_films = ...
# YOUR CODE HERE
def combine_all_dataframes(rated_nominees_df, grossing_budget_df):
    # Combine dataframes using an inner join
    top_films = pd.merge(rated_nominees_df, grossing_budget_df, how='inner', on='film')
    
    return top_films


top_films = combine_all_dataframes(best_rated_nominees, top_grossing_budget)

In [16]:
assert top_films.shape == (184, 14)
assert set(top_films.columns) == set(
    [
       'year_film', 'year_ceremony', 'category', 'name', 'film', 'winner',
       'runtime', 'rating', 'metascore', 'no_votes', 'budget_rank',
       'production_cost', 'gross_rank', 'gross'
    ]
)

## 7 Final transformations (graded)

Now, we perform the train-test split.

We also perform some final transformations on both datasets:

* Tranform "winner" into a binary feature
* Create a new feature, rating_rank, from the rating column. 
* Filter out the movies that appear less than 10 times in the DataFrame.
* Keep only 'film', 'winner', 'category', 'runtime', 'rating', 'metascore', 'no_votes', 'budget_rank', 'production_cost', 'gross_rank', 'rating_rank' and 'gross' columns.

In [17]:
def transform_winner(df):
    df = df.copy()
    # df = ...
    # YOUR CODE HERE
    df['winner'] = df['winner'].apply(lambda x: 1 if x == 'Yes' else 0)
    return df

def create_rating_rank(df):
    df = df.copy()
    # df['rating_rank'] = 
    # YOUR CODE HERE
    df['rating_rank'] = df['rating'].rank(ascending=True)
    return df

def preprocess_data(df):
    # You should follow these exact steps:
    #   1 - Binarize 'winner'
    #   2 - Filter out rows that have a film that appear is less than 10 times in the DataFrame.
    #   3 - Keep only the 'film', 'winner', category', 'runtime', 'rating', 'metascore', 'no_votes', 
    #      'budget_rank', 'production_cost', 'gross_rank', 'rating_rank' and 'gross' columns
    #   5 - Create a new feature, rating_rank, from the ranking column. 
    #   6 - Sort the DataFrame by 'rating' in ascending order
    # YOUR CODE HERE
    df = df.copy()

    # Binarize 'winner'
    df = transform_winner(df)

    # Filter out rows that have a film that appears less than 10 times
    film_count = df['film'].value_counts()
    df = df[df['film'].isin(film_count.index[film_count >= 10])]

    # Keep only the specified columns
    columns_to_keep = ['film', 'winner', 'category', 'runtime', 'rating', 'metascore', 
                       'no_votes', 'budget_rank', 'production_cost', 'gross_rank', 
                       'gross']
    df = df[columns_to_keep]

    # Create 'rating_rank' and sort by 'rating'
    df = create_rating_rank(df).sort_values('rating', ascending=True)

    return df



top_films_preprocessed = preprocess_data(top_films)

In [18]:
assert top_films_preprocessed.shape == (59, 12)
assert set(top_films_preprocessed.columns) == {
       'category', 'film', 'runtime', 'rating', 'metascore', 'no_votes',
       'budget_rank', 'production_cost', 'gross_rank', 'gross', 'rating_rank','winner'
}
assert top_films_preprocessed.budget_rank.min() == 49
assert top_films_preprocessed.gross_rank.max() == 180
assert top_films_preprocessed.iloc[0].no_votes == 769145

# The house prices dataset

A dataset containing several characteristics of several houses and their selling price 

* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* OverallQual: Rates the overall material and finish of the house
       10  Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
* OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
* MasVnrArea: Masonry veneer area in square feet
* BsmtFinSF1: Type 1 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* BedroomAbvGr: Bedrooms above grade (does NOT include basement bedrooms)
* KitchenAbvGr: Kitchens above grade
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Fireplaces: Number of fireplaces
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* MiscVal: $Value of miscellaneous feature 
* SellingDate: Date when the house was sold
* BuildingDate: Date when the house was built
* RemodAddDate: Remodel date (same as construction date if no remodeling or additions)
* SalePrice: The house price at the selling date (our target variable)

Let's read the csv and create our train-test-split

In [19]:
def house_price_dataset():
    return pd.read_csv(
    'data/housePrices.csv', 
        parse_dates=[
            'SellingDate',
            'BuildingDate',
            'RemodAddDate'
        ]
    )

dataset = house_price_dataset()
dataset_train, dataset_test = train_test_split(dataset, random_state=0)
X_train = dataset_train.drop(columns='SalePrice')
y_train = dataset_train.SalePrice
X_test = dataset_test.drop(columns='SalePrice')
y_test = dataset_test.SalePrice

## 8 Build a DateTransformer transformer (graded)

A simple transformer that transforms dates into timedeltas can be useful, from times to times, when modeling. Usually when you have features that are Dates you compute a time delta between the feature and a given refence date.

e.g Imagine that your clients have a loyalty period that ends at a given date. When your model is doing some predictions, one of the features that you can use is the number of days until the end of the loyalty period. i.e the date when the loyalty ends minus the date when your model is running. 

In the house prices dataset, the selling date will be the reference data, since we want to predict the house price at the selling date. For instance, two houses with the exact same features can vary in prices if the construction year is different. So we should input this information and feed into the model. Then we need to convert the other dates using our transformer

Hint: Result should be integers

In [20]:
#class DateTransformer(BaseEstimator, TransformerMixin):
#    # Implement the __init__ method.
#    # Our DateTransformer must be able to receive two parameters: 
#    # datetime_cols: a list, that contains the datetime cols that should be converted
#    # ref_date_col: indicates the name of the column that should be used as reference date,
#    # YOUR CODE HERE
#    def __init__(self, datetime_cols: List[str], ref_date_col: str) -> None:
#        # Initialize with datetime columns and reference date column
#        self.datetime_cols = datetime_cols
#        self.ref_date_col = ref_date_col
#        
#    # There's no need for a fit method in this case, it does nothing.
#    # We should be able to call fit without any explicit parameters.
#    # Meaning: we should be able to call fit() on the transformer.
#    # YOUR CODE HERE
#    def fit(self, X: pd.DataFrame, y=None)-> None:
#        # Fit does nothing in this case, just returns self
#        return self
#
#    # Transform should transform all datetime columns into the difference in days to the reference date.
#    # The reference date column should be dropped. 
#    # YOUR CODE HERE
#    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
#        # Ensure working on a copy of the data to avoid altering original dataframe
#        X_transformed = X.copy()
#
#        # Check if the reference date column exists
#        if self.ref_date_col not in X_transformed.columns:
#            raise ValueError(f"Reference date column '{self.ref_date_col}' not found in the dataframe.")
#
#        # Convert each datetime column to timedelta in days relative to the reference date
#        for col in self.datetime_cols:
#            if col not in X_transformed.columns:
#                raise ValueError(f"Column '{col}' not found in the dataframe.")
#
#            X_transformed[col] = (X_transformed[self.ref_date_col] - X_transformed[col]).dt.days
#        
#        # Drop the reference date column
#        X_transformed.drop(columns=[self.ref_date_col], inplace=True)
#
#        # Convert to int64
#        for col in self.datetime_cols:
#            X_transformed[col] = X_transformed[col].astype(np.int64)
#
#        return X_transformed

In [21]:
from typing import List


class DateTransformer(BaseEstimator, TransformerMixin):
    # Implement the __init__ method.
    # Our DateTransformer must be able to receive two parameters: 
    # datetime_cols: a list, that contains the datetime cols that should be converted
    # ref_date_col - indicates the name of the column that should be used as reference date,
    # YOUR CODE HERE
    # raise NotImplementedError()
    def __init__(self, datetime_cols: List[str], ref_date_col: str) -> None:
        self.datetime_cols = datetime_cols
        self.ref_date_col = ref_date_col
        
        
    # There's no need for a fit method in this case, it does nothing.
    # We should be able to call fit without any explicit parameters.
    # Meaning: we should be able to call transformer.fit().
    # YOUR CODE HERE
    # raise NotImplementedError()
    def fit(self, X: pd.DataFrame, y=None)-> None:
        return self

    # Transform should transform all datetime columns into the difference in days to the reference date.
    # The reference date column should be dropped. 
    # YOUR CODE HERE
    # raise NotImplementedError()
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        for datetime_col in self.datetime_cols:
            X[datetime_col] = (pd.to_datetime(X[datetime_col]) - pd.to_datetime(X[self.ref_date_col])).dt.days.astype(int)
        return X.drop(self.ref_date_col, axis=1)


In [22]:
X_train_transformed = DateTransformer(
    datetime_cols=['BuildingDate', 'RemodAddDate'], 
    ref_date_col='SellingDate'
).fit_transform(X_train)
assert X_train_transformed.BuildingDate.min() == -49008
assert X_train_transformed.BuildingDate.max() == -1
assert 'SellingDate' not in X_train_transformed.columns
assert X_train_transformed.dtypes.BuildingDate == np.dtype('int64')
assert X_train_transformed.dtypes.RemodAddDate == np.dtype('int64')

You might be wondering why we have to implement it as a Transformer Class, and not using functions.
You'll understand the reason in the next section, we're we can tie them all together in a `Pipeline`.

## 9 Building the pipeline (graded)

Finally, we want to use the two transformers together and run a linear regression on top. We want to:

* Convert the dates to time deltas relative to the Selling Date.

* Scale all features to the same range, using `sklearn.preprocessing.RobustScaler()`.

* Estimate the SellingPrice using a Linear Regression.

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data (i.e., Gaussian with zero mean and unit variance).

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.



In [23]:
X_train.describe()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SellingDate,BuildingDate,RemodAddDate
count,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,...,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051,1051,1051
mean,56.516651,10165.033302,6.095147,5.594672,102.087536,564.330162,1046.601332,1159.809705,351.509039,6.424358,...,97.315890,44.084681,21.963844,3.058991,15.811608,2.394862,28.690771,2007-11-05 04:29:54.862035968,1970-06-04 20:34:28.886774500,1984-04-20 15:15:14.557564224
min,0.000000,1491.000000,1.000000,2.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2006-01-01 00:00:00,1875-11-01 00:00:00,1950-01-01 00:00:00
25%,40.000000,7500.000000,5.000000,5.000000,0.000000,217.500000,792.500000,876.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2007-01-04 00:00:00,1953-01-11 00:00:00,1967-01-03 00:00:00
50%,61.000000,9505.000000,6.000000,5.000000,0.000000,463.000000,990.000000,1077.000000,0.000000,0.000000,...,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2008-01-05 00:00:00,1972-01-09 00:00:00,1992-01-12 00:00:00
75%,78.000000,11635.000000,7.000000,6.000000,161.000000,808.000000,1282.500000,1382.500000,736.000000,0.000000,...,177.500000,64.500000,0.000000,0.000000,0.000000,0.000000,0.000000,2009-01-06 00:00:00,1999-01-07 00:00:00,2003-01-05 00:00:00
max,182.000000,115149.000000,10.000000,9.000000,1600.000000,2336.000000,3200.000000,3228.000000,2065.000000,572.000000,...,728.000000,547.000000,386.000000,508.000000,480.000000,738.000000,3500.000000,2010-01-07 00:00:00,2009-01-12 00:00:00,2010-01-04 00:00:00
std,33.228422,6319.536187,1.365732,1.120939,179.723470,442.621653,418.210100,380.318077,437.579545,52.580304,...,124.988613,62.975199,60.397025,27.657847,56.470192,39.015517,190.231875,NaN,NaN,NaN


In [24]:
# Create a pipeline including:
#   1 - 'date_converter', DateTransformer(['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')
#   2 - 'robust_scaler', RobustScaler() with the default parameters
#   3 - 'model', LinearRegression
# YOUR CODE HERE
pipeline = Pipeline(
        [
        ('date_converter', DateTransformer(datetime_cols=['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')),
        ('robust_scaler', RobustScaler()),
        ('model', LinearRegression())
        ]
    )


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print('MAE: {}'.format(mae))

MAE: 20737.06019314779


In [25]:
assert type(pipeline) == Pipeline
assert type(pipeline.named_steps['date_converter']) == DateTransformer
assert type(pipeline.named_steps['robust_scaler']) == RobustScaler
assert pipeline.named_steps['date_converter'].get_params()['ref_date_col'] == 'SellingDate'
assert set(
    pipeline.named_steps['date_converter'].get_params()['datetime_cols']
) == {'BuildingDate', 'RemodAddDate'}
assert type(pipeline.named_steps['model']) == LinearRegression 

## 10. Access the cofficients from the pipeline (ungraded)

Now we would want to obtain the coefficients from the model to understand features with the most predictive power.

In [26]:
#coefs = ....
# YOUR CODE HERE
coefs = pipeline.named_steps['model'].coef_

In [27]:
assert coefs.shape == (30,), 'Wrong number of coefficients. Did you select the features correctly?'

Exercises complete, congratulations! You are about to become a certified data wrangler.